In [1]:
import tensorflow as tf

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

import numpy as np
import pandas as pd

items = pd.read_csv(r"..\Data\Raw\items.csv", delimiter=",")
print(items.head())
print(items.shape)
items = items.drop_duplicates(subset=['item_name'], keep=False)
print(items.shape)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
                                           item_name  item_id  \
0          ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D        0   
1  !ABBYY FineReader 12 Professional Edition Full...        1   
2      ***В ЛУЧАХ СЛАВЫ   (UNV)                    D        2   
3    ***ГОЛУБАЯ ВОЛНА  (Univ)                      D        3   
4        ***КОРОБКА (СТЕКЛО)                       D        4   

   item_category_id  
0                40  
1                76  
2                40  
3                40  
4                40  
(22170, 3)
(22170, 3)


In [17]:
test = pd.read_csv(r"..\Data\Raw\test.csv", delimiter=",")
print(test.head())
print(test['shop_id'].nunique())
print(test['ID'].nunique())

   ID  shop_id  item_id
0   0        5     5037
1   1        5     5320
2   2        5     5233
3   3        5     5232
4   4        5     5268
42
214200


In [18]:
sales = pd.read_csv(r"..\Data\Raw\sales_train.csv", delimiter=",")
print(sales.head())
print(sales.shape)
sales = sales.drop_duplicates(subset=['date', 'shop_id', 'item_id'], keep=False)
print(sales.shape)
sales = sales[sales.item_cnt_day<1000]
sales = sales[sales.item_price<60000]
sales = sales[sales.item_price>0]
print(sales.shape)
sales['revenue'] = sales['item_cnt_day'] * sales['item_price']
sales_monthly = sales.groupby(["date_block_num", "shop_id", "item_id"]).agg({'item_cnt_day': ['sum'], 'item_price':['median'], 'revenue':['sum']})
print(sales_monthly.head(20))
sales_monthly.sample(10)

         date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0  02.01.2013               0       59    22154      999.00           1.0
1  03.01.2013               0       25     2552      899.00           1.0
2  05.01.2013               0       25     2552      899.00          -1.0
3  06.01.2013               0       25     2554     1709.05           1.0
4  15.01.2013               0       25     2555     1099.00           1.0
(2935849, 6)
(2935793, 6)
(2935789, 6)
                               item_cnt_day item_price revenue
                                        sum     median     sum
date_block_num shop_id item_id                                
0              0       32               6.0      221.0  1326.0
                       33               3.0      347.0  1041.0
                       35               1.0      247.0   247.0
                       43               1.0      221.0   221.0
                       51               2.0      128.5   257.0
             

,,,item_cnt_day,item_price,revenue
,,,sum,median,sum
date_block_num,shop_id,item_id,,,
8,18,1467,2.0,899.0,1798.0
12,27,12107,1.0,499.0,499.0
19,18,6111,1.0,74.0,74.0
10,45,4475,1.0,599.0,599.0
9,22,7070,2.0,599.0,1198.0
33,53,2652,1.0,349.0,349.0
28,25,253,1.0,249.0,249.0
8,31,13955,1.0,249.0,249.0


In [19]:
#join to test set to get only necessary IDs
month_range = range(1, 34)
month_df = pd.DataFrame(month_range)

month_df.rename(columns={0 :'date_block_num'}, inplace=True )
print(month_df.head())
month_df['temp'] = 1
test['temp'] = 1
test_monthly = test.merge(month_df, on=['temp'])
test_monthly.drop(['temp'], axis=1, inplace=True)
whole_df = test_monthly.merge(sales_monthly, how='left', on=['date_block_num', 'item_id', 'shop_id'])
whole_df.sample(10)
print(whole_df.shape)

   date_block_num
0               1
1               2
2               3
3               4
4               5
(7068600, 7)


e:\latest pycharm\venv\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


In [20]:
categories = pd.read_csv(r"..\Data\Raw\item_categories.csv", delimiter=",")
print(categories.head())


        item_category_name  item_category_id
0  PC - Гарнитуры/Наушники                 0
1         Аксессуары - PS2                 1
2         Аксессуары - PS3                 2
3         Аксессуары - PS4                 3
4         Аксессуары - PSP                 4


In [21]:
print(items.shape)
print(categories.shape)
item_cats = items.merge(categories, how='left', on=['item_category_id'])
print(item_cats.shape)
print(item_cats.head())
item_cats.drop(['item_category_name', 'item_name'], axis=1, inplace=True)
print(item_cats.head())
print(item_cats['item_id'].nunique())
print(item_cats.shape)
whole_df = whole_df.merge(item_cats, how='left', on=['item_id'])
whole_df.sample(10)

(22170, 3)
(84, 2)
(22170, 4)
                                           item_name  item_id  \
0          ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D        0   
1  !ABBYY FineReader 12 Professional Edition Full...        1   
2      ***В ЛУЧАХ СЛАВЫ   (UNV)                    D        2   
3    ***ГОЛУБАЯ ВОЛНА  (Univ)                      D        3   
4        ***КОРОБКА (СТЕКЛО)                       D        4   

   item_category_id                    item_category_name  
0                40                            Кино - DVD  
1                76  Программы - Для дома и офиса (Цифра)  
2                40                            Кино - DVD  
3                40                            Кино - DVD  
4                40                            Кино - DVD  
   item_id  item_category_id
0        0                40
1        1                76
2        2                40
3        3                40
4        4                40
22170
(22170, 2)


,ID,shop_id,item_id,date_block_num,"(item_cnt_day, sum)","(item_price, median)","(revenue, sum)",item_category_id
6632069,200971,44,21355,27,NaN,NaN,NaN,38
4194885,127117,52,2722,25,NaN,NaN,NaN,31
1826082,55335,26,11280,28,NaN,NaN,NaN,31
5926160,179580,38,7982,21,NaN,NaN,NaN,16
66672,2020,5,21827,13,NaN,NaN,NaN,41
4838368,146617,58,20726,8,NaN,NaN,NaN,72
2324172,70429,24,18327,16,NaN,NaN,NaN,57
6709507,203318,44,7088,14,NaN,NaN,NaN,31
6622675,200687,44,11930,5,NaN,NaN,NaN,40
1070855,32450,10,14124,6,NaN,NaN,NaN,41


In [22]:
whole_df.fillna(0, inplace=True)
list(whole_df.columns)
whole_df.rename(columns={('item_cnt_day', 'sum'):'total_count', ('item_price', 'median'): 'median_price', ('revenue', 'sum'):'total_revenue'}, inplace=True )
whole_df['total_count'].clip(0,20)
print(whole_df.sample(10))
whole_df.to_csv('../Data/Processed/pre_normalised_whole_df.csv', index=False)

             ID  shop_id  item_id  date_block_num  total_count  median_price  \
3239292   98160       19     7133              13          0.0           0.0   
3995725  121082       53    17996              20          0.0           0.0   
930869    28208        7     3023               6          0.0           0.0   
5551288  168220       36    16057              29          0.0           0.0   
3747026  113546       49      938               9          0.0           0.0   
1274111   38609       12    21118              15          0.0           0.0   
6983854  211631       45     6311              32          0.0           0.0   
3165010   95909       14    19771              14          1.0         399.0   
5792891  175542       35    13058               6          0.0           0.0   
569221    17249        3     8607               5          0.0           0.0   

         total_revenue  item_category_id  
3239292            0.0                19  
3995725            0.0           

In [23]:
whole_df.sort_values(by=['ID', 'date_block_num'])
whole_df = whole_df[whole_df['item_id'] <= 1000]
whole_df.drop(['ID', 'total_revenue'], axis=1, inplace=True)
whole_df = whole_df[['total_count', 'date_block_num', 'shop_id', 'item_id', 'item_category_id', 'median_price']]
print(whole_df.head(34))
whole_df_arr = whole_df.values
print(whole_df_arr.shape)
whole_df_arr = whole_df_arr.reshape(7350, 33, 6)
print(whole_df_arr)#change back to 214200


       total_count  date_block_num  shop_id  item_id  item_category_id  \
15774          0.0               1        5      560                77   
15775          0.0               2        5      560                77   
15776          0.0               3        5      560                77   
15777          0.0               4        5      560                77   
15778          0.0               5        5      560                77   
15779          0.0               6        5      560                77   
15780          0.0               7        5      560                77   
15781          0.0               8        5      560                77   
15782          1.0               9        5      560                77   
15783          0.0              10        5      560                77   
15784          0.0              11        5      560                77   
15785          0.0              12        5      560                77   
15786          0.0              13    

In [24]:
#prepare training and validation sets
train_arr = whole_df_arr[:, :26]
val_arr = whole_df_arr[:, 26:]
print(train_arr.shape)
print(val_arr.shape)
train_arr_first_test = train_arr[:, 19:25]
print(train_arr_first_test.shape)
print(train_arr)

(7350, 26, 6)
(7350, 7, 6)
(7350, 6, 6)
[[[0.00e+00 1.00e+00 5.00e+00 5.60e+02 7.70e+01 0.00e+00]
  [0.00e+00 2.00e+00 5.00e+00 5.60e+02 7.70e+01 0.00e+00]
  [0.00e+00 3.00e+00 5.00e+00 5.60e+02 7.70e+01 0.00e+00]
  ...
  [0.00e+00 2.40e+01 5.00e+00 5.60e+02 7.70e+01 0.00e+00]
  [0.00e+00 2.50e+01 5.00e+00 5.60e+02 7.70e+01 0.00e+00]
  [0.00e+00 2.60e+01 5.00e+00 5.60e+02 7.70e+01 0.00e+00]]

 [[0.00e+00 1.00e+00 5.00e+00 5.62e+02 7.70e+01 0.00e+00]
  [0.00e+00 2.00e+00 5.00e+00 5.62e+02 7.70e+01 0.00e+00]
  [0.00e+00 3.00e+00 5.00e+00 5.62e+02 7.70e+01 0.00e+00]
  ...
  [0.00e+00 2.40e+01 5.00e+00 5.62e+02 7.70e+01 0.00e+00]
  [0.00e+00 2.50e+01 5.00e+00 5.62e+02 7.70e+01 0.00e+00]
  [0.00e+00 2.60e+01 5.00e+00 5.62e+02 7.70e+01 0.00e+00]]

 [[0.00e+00 1.00e+00 5.00e+00 8.39e+02 7.30e+01 0.00e+00]
  [0.00e+00 2.00e+00 5.00e+00 8.39e+02 7.30e+01 0.00e+00]
  [1.00e+00 3.00e+00 5.00e+00 8.39e+02 7.30e+01 3.30e+03]
  ...
  [0.00e+00 2.40e+01 5.00e+00 8.39e+02 7.30e+01 0.00e+00]
  [0.00e+0

In [25]:
#normalise data

train_mean = train_arr.mean(axis=(0, 1))
train_std = train_arr.std(axis=(0, 1))
train_arr = (train_arr - train_mean)/train_std
print(train_arr)
val_arr = (val_arr - train_mean)/train_std


[[[-0.09934477 -1.66666667 -1.51708349  0.10151063  1.19542789
   -0.1184017 ]
  [-0.09934477 -1.53333333 -1.51708349  0.10151063  1.19542789
   -0.1184017 ]
  [-0.09934477 -1.4        -1.51708349  0.10151063  1.19542789
   -0.1184017 ]
  ...
  [-0.09934477  1.4        -1.51708349  0.10151063  1.19542789
   -0.1184017 ]
  [-0.09934477  1.53333333 -1.51708349  0.10151063  1.19542789
   -0.1184017 ]
  [-0.09934477  1.66666667 -1.51708349  0.10151063  1.19542789
   -0.1184017 ]]

 [[-0.09934477 -1.66666667 -1.51708349  0.1082948   1.19542789
   -0.1184017 ]
  [-0.09934477 -1.53333333 -1.51708349  0.1082948   1.19542789
   -0.1184017 ]
  [-0.09934477 -1.4        -1.51708349  0.1082948   1.19542789
   -0.1184017 ]
  ...
  [-0.09934477  1.4        -1.51708349  0.1082948   1.19542789
   -0.1184017 ]
  [-0.09934477  1.53333333 -1.51708349  0.1082948   1.19542789
   -0.1184017 ]
  [-0.09934477  1.66666667 -1.51708349  0.1082948   1.19542789
   -0.1184017 ]]

 [[-0.09934477 -1.66666667 -1.517083

In [26]:
#train_arr = list(train_arr)

#train_arr = train_arr[0:3]
train_list = list(train_arr)
val_list = list(val_arr)
first_test_list = list(train_arr_first_test)
#def split_sequences(sequences, n_steps):
#    pass

In [27]:
#get sliding windows for each ID
train_x, train_y = [],[]
val_x, val_y = [],[]
def get_windows(sequence, n_steps, x, y):
    for i in range(len(sequence)):
    # find the end of this pattern
        end_ix = i + n_steps
    # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
    # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix][0]
        x.append(seq_x)
        y.append(seq_y)

for example in train_list:
    get_windows(example, 6, train_x, train_y)
for example in val_list:
    get_windows(example, 6, val_x, val_y)
print(val_y[0])

train_x = np.asarray(train_x)
train_y = np.asarray(train_y)
val_x = np.asarray(val_x)
val_y = np.asarray(val_y)
print(train_x.shape)
print(val_y.shape)


-0.09934476591470832
(147000, 6, 6)
(7350,)


In [28]:
shop_id_x = train_x[:, :, 2]

shop_id_val_x = val_x[:, :, 2]
train_x = np.delete(train_x, 2, axis=2)
val_x = np.delete(val_x, 2, axis=2)
print(train_x.shape)


shop_id_x = shop_id_x*train_std[2] + train_mean[2]
shop_id_val_x = shop_id_val_x*train_std[2] + train_mean[2]
print(shop_id_x.shape)

shop_id_input = tf.keras.Input(shape=(6))
numerical_input = tf.keras.Input(shape=(6, 5))
shop_id_embed = tf.keras.layers.Embedding(input_dim=500, input_length=(6,1), output_dim=8, name="Shop ID embed")(shop_id_input)
#shop_id_flatten = tf.reshape(shop_id_embed, shape=(6, 8))
#shop_id_flatten = tf.keras.layers.Flatten()(shop_id_embed)
combine_input = tf.keras.layers.Concatenate()([shop_id_embed, numerical_input])
model_shop_id = tf.keras.Model([shop_id_input, numerical_input], combine_input)
print(combine_input.shape)
from keras.utils import plot_model
plot_model(model_shop_id, to_file='model.png')
model_shop_id.summary()

(147000, 6, 5)
(147000, 6)
(None, 6, 13)
Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 6)]          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 6, 8)         4000        input_3[0][0]                    
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 6, 5)]       0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 6, 13)        0           embedding_1[0][0]                
                                              

In [29]:
#integer_arr = tf.constant([[1],[2],[3]])
#embedding_layer = tf.keras.layers.Embedding(100, 3)
#print(shop_id_x)
#shop_id_embed = tf.keras.layers.Embedding(input_dim=1000, input_length=(6,1), output_dim=8)(shop_id_x)
#combine_input = tf.keras.layers.Concatenate()([shop_id_embed, train_x])
#print(combine_input)
#print(combine_input.shape)

In [30]:
lstm_1 = tf.keras.layers.LSTM(12)(combine_input)
dense_1 = tf.keras.layers.Dense(1)(lstm_1)
total_model = tf.keras.Model([shop_id_input, numerical_input], dense_1)


total_model.compile(loss="mse", optimizer="adam")

history = total_model.fit(x=[shop_id_x, train_x], y=train_y, epochs=100, batch_size = 1, verbose=1)



plot_model(total_model, to_file='master_model.png')
total_model.summary()

Epoch 1/100
126061/147000 [========================>.....] - ETA: 52s - loss: 0.5867     

KeyboardInterrupt: 

In [ ]:
#model = tf.keras.models.Sequential([
 #      tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(12, input_shape=(6,6))),
 #   tf.keras.layers.Dense(1)
#])

#model.compile(loss="mse", optimizer="adam")
#history = model.fit(x=train_x, y=train_y, validation_data=(val_x, val_y), epochs=100, batch_size=40, verbose=1)


In [ ]:
#print(val_arr)
#print(train_arr[:,-7:-1])
#create prediction window excluding last value to compare
predictions = model.predict(val_arr[:,:-1])
predictions = model.predict(train_arr[:,-7:-1])
denormalised_predictions = predictions*train_std[0] + train_mean[0]

In [ ]:

print(denormalised_predictions.astype(int))
print(train_mean)
print(train_std)